# Q-Learning

In [1]:
import numpy as np
import random

In [4]:
np.zeros((4*4,4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [2]:
R = np.array([[0, 0, 0, -1, 0, 0, -1, 0, 0, 0],
                     [0, -1, -1, -1, 0, 0, 0, 0, -1, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
                     [0, -1, 0, 0, -1, 0, 0, 0, -1, 0],
                     [0, -1, 0, 0, -1, -1, -1, -1, -1, 100],
                     [0, -1, 0, 0, 0, 0, 0, 0, -1, 0],
                     [0, -1, -1, -1, 0, 0, -1, 0, -1, 0],
                     [0, 0, 0, -1, 0, 0, -1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, -1, 0, 0, 0]])

In [72]:
alpha = 0.7
gamma = 0.95
max_epsilon = 1.0
min_epsilon = 0.05
decay_rate = 0.0005
epochs = 20000
actions = ['up', 'down', 'left', 'right']
x = len(R)
y = len(R[0])
states = x*y
Q = np.zeros((x*y, 4))
max_steps = 2000
goal_state = 49

In [73]:
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


In [74]:
state = 0
walls = []

for i in range(x):
    for j in range(y):
        if R[i][j] == -1:
            walls.append(state)
            for a in range(len(actions)):
                Q[state][a] = -1
        state += 1

state = 0

for i in range(x):
    for j in range(y):
        if i == 0 or state - 10 in walls:
            Q[state][actions.index('up')] = -1
        if i == x - 1 or state + 10 in walls:
            Q[state][actions.index('down')] = -1
        if j == 0 or state - 1 in walls:
            Q[state][actions.index('left')] = -1
        if j == y - 1 or state + 1 in walls:
            Q[state][actions.index('right')] = -1
        state += 1

In [75]:
Q

array([[-1.,  0., -1.,  0.],
       [-1., -1.,  0.,  0.],
       [-1., -1.,  0., -1.],
       [-1., -1., -1., -1.],
       [-1.,  0., -1.,  0.],
       [-1.,  0.,  0., -1.],
       [-1., -1., -1., -1.],
       [-1.,  0., -1.,  0.],
       [-1., -1.,  0.,  0.],
       [-1.,  0.,  0., -1.],
       [ 0.,  0., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [ 0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1., -1., -1., -1.],
       [ 0.,  0., -1., -1.],
       [ 0.,  0., -1.,  0.],
       [-1., -1.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1., -1., -1., -1.],
       [ 0.,  0., -1., -1.],
       [ 0.,  0., -1., -1.],
       [-1., -1., -1., -1.],
       [ 0.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1., -

In [76]:
def get_epsilon(epoch):
    return min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * epoch)

In [77]:
def take_action(random_state, epsilon):
    indexes = [index for index,value in enumerate(Q[random_state]) if value != -1]
    possible_actions = [value for index,value in enumerate(actions) if index in indexes]
    if np.random.random() > epsilon:
        if np.max(Q[random_state]) == -1:
            return None
        else:
            return actions[np.argmax(Q[random_state])]
    else:    
        return random.choice(possible_actions)

In [78]:
def update_table(state, action, reward, next_state):
    """
    Update Q table
    """
    Q[state][actions.index(action)] += alpha * (reward + gamma * np.max(Q[next_state]))

In [79]:
possible_states = [x for x in range(len(Q)) if x not in walls]
for epoch in range(epochs):
    random_state = random.choice(possible_states)
    epsilon = get_epsilon(epoch)
    for steps in range(max_steps):
        action = take_action(random_state, epsilon)
        if action == None:
            break
        elif action == 'up':
            next_state = random_state - 10
        elif action == 'down':
            next_state = random_state + 10
        elif action == 'left':
            next_state = random_state - 1
        elif action == 'right':
            next_state = random_state + 1
        if random_state != goal_state:
            reward = -1
        elif random_state == goal_state:
            reward = 1
            break
        #print(f"Epoch: {epoch}, steps: {steps}, random state: {random_state}, next state: {next_state}")
        update_table(random_state, action, reward, next_state)
        random_state = next_state

In [80]:
Q

array([[-1.00000000e+00, -1.88170500e+02, -1.00000000e+00,
        -2.05450000e+02],
       [-1.00000000e+00, -1.00000000e+00, -2.18200500e+02,
        -2.20265500e+02],
       [-1.00000000e+00, -1.00000000e+00, -2.57355000e+02,
        -1.00000000e+00],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00],
       [-1.00000000e+00, -1.80180000e+02, -1.00000000e+00,
        -1.60272000e+02],
       [-1.00000000e+00, -9.43405815e+03, -1.63135000e+02,
        -1.00000000e+00],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00],
       [-1.00000000e+00, -1.32205500e+02, -1.00000000e+00,
        -1.58840500e+02],
       [-1.00000000e+00, -1.00000000e+00, -1.39930000e+02,
        -1.44725000e+02],
       [-1.00000000e+00, -1.43566500e+02, -1.22255000e+02,
        -1.00000000e+00],
       [-1.93200000e+02, -1.43125500e+02, -1.00000000e+00,
        -1.00000000e+00],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
      

In [85]:
random_state = random.choice(possible_states)

In [86]:
random_state

64

In [93]:
Q[55]

array([-108.3355,  -96.985 ,  -98.98  ,   -1.    ])

In [88]:
np.max(Q[random_state])

-1.0

In [ ]:
while random_state != goal_state:
    